# Vorarbeit

In [33]:
import pm4py
import keras
import pandas as pd
import numpy as np
from pm4py.objects.conversion.log import converter as log_converter
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

from keras.models import load_model
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Reshape, GaussianNoise
from keras.utils.vis_utils import plot_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [34]:
event_log = pm4py.read_csv("final_dataset.csv")

df_labour = log_converter.apply(event_log, variant=log_converter.Variants.TO_DATA_FRAME)


df = df_labour
df = df[['id', 'action', 'duration']]
df.columns = ['action', 'id', 'duration']
df.to_csv("final_dataset.csv", index = False)
df = pd.read_csv('adl_log.csv', parse_dates = ['time'], dtype = {'action' : 'category', 'status' : 'category', 'id' : int})
print(df)


               action    status                      time   id
0               Start     start 2011-06-15 00:33:36+02:00   11
1               Start  complete 2011-06-15 00:33:36+02:00   11
2     personalhygiene     start 2011-06-15 00:33:36+02:00   11
3     personalhygiene  complete 2011-06-15 00:38:44+02:00   11
4               sleep     start 2011-06-15 00:45:24+02:00   11
...               ...       ...                       ...  ...
4195            sleep  complete 2011-08-12 23:48:14+02:00  591
4196           toilet     start 2011-08-12 23:48:15+02:00  591
4197           toilet  complete 2011-08-13 00:11:26+02:00  591
4198              End     start 2011-08-13 00:11:26+02:00  591
4199              End  complete 2011-08-13 00:11:26+02:00  591

[4200 rows x 4 columns]


In [35]:
timediffs = []
for i in range(1, len(df), 2):
    diff = df.iloc[i, 2] - df.iloc[i-1, 2]
    timediffs.append(diff)
    timediffs.append(diff)

df = df.sort_values(by=['id','time'])
df['duration'] = timediffs
df = df.iloc[::2, [0,3,4]].reset_index(drop=True)

# Listen erstellen

In [36]:
index_of_bathe=df[df ["action"] == "bathe"].index.to_list()
index_of_bathe



[1086, 1256, 1302, 1366, 1574, 1609, 1683, 1810, 1836, 1968]

In [37]:
vorgänger_bathe=[]
for i in index_of_bathe:
    vorgänger_bathe.append(i-1)
    
    
nachfolger_bathe=[]
for i in index_of_bathe:
    nachfolger_bathe.append(i+1)

vorgänger_bathe

[1085, 1255, 1301, 1365, 1573, 1608, 1682, 1809, 1835, 1967]

In [38]:
df.iloc[vorgänger_bathe]

,action,id,duration
1085,Start,341,00:00:00
1255,sleep,371,01:00:29
1301,sleep,381,01:05:10
1365,cleaning,411,00:18:37
1573,medication,451,00:01:09
1608,sleep,481,00:50:30
1682,Start,501,00:00:00
1809,sleep,521,00:00:28
1835,watchtv,521,00:21:48
1967,personalhygiene,571,00:06:20


In [39]:
df.iloc[nachfolger_bathe]

,action,id,duration
1087,dress,341,00:00:41
1257,dress,371,00:07:13
1303,dress,381,00:01:46
1367,dress,411,00:01:36
1575,dress,451,00:00:36
1610,dress,481,00:01:43
1684,dress,501,00:00:19
1811,dress,521,00:03:21
1837,dress,521,00:00:41
1969,groom,571,00:00:42


In [40]:
index_of_cleaning=df[df ["action"] == "cleaning"].index.to_list()

vorgänger_cleaning=[]
for i in index_of_cleaning:
    vorgänger_cleaning.append(i-1)
    
nachfolger_cleaning=[]
for i in index_of_cleaning:
    nachfolger_cleaning.append(i+1)

df.iloc[nachfolger_cleaning]

,action,id,duration
7,dress,11,00:19:06
31,watchtv,21,03:56:49
72,medication,31,00:00:06
110,work,61,00:00:34
123,personalhygiene,61,00:00:13
...,...,...,...
1965,toilet,571,00:02:14
2023,work,581,00:18:53
2030,work,581,00:14:30
2075,personalhygiene,591,00:10:37


In [41]:
index_of_dress=df[df ["action"] == "dress"].index.to_list()

vorgänger_dress=[]
for i in index_of_dress:
    vorgänger_dress.append(i-1)
    
nachfolger_dress=[]
for i in index_of_dress:
    nachfolger_dress.append(i+1)

df.iloc[vorgänger_dress]
df.iloc[nachfolger_dress]


,action,id,duration
8,watchtv,11,00:50:36
19,outdoors,11,00:19:22
28,End,11,00:00:00
38,work,21,00:10:23
1046,toilet,311,00:06:08
1064,sleep,311,00:43:46
1079,personalhygiene,311,00:14:48
1088,medication,341,00:02:57
1142,toilet,351,00:02:51
1144,toilet,351,00:05:29


In [42]:
index_of_eatingdrinking=df[df ["action"] == "eatingdrinking"].index.to_list()

vorgänger_eatingdrinking=[]
for i in index_of_eatingdrinking:
    vorgänger_eatingdrinking.append(i-1)
    
nachfolger_eatingdrinking=[]
for i in index_of_eatingdrinking:
    nachfolger_eatingdrinking.append(i+1)

df.iloc[vorgänger_eatingdrinking]
df.iloc[nachfolger_eatingdrinking]


,action,id,duration
6,cleaning,11,00:01:01
27,dress,11,01:56:24
41,medication,21,00:03:33
45,snack,21,00:22:24
61,relax,21,00:40:54
...,...,...,...
1916,cleaning,561,00:04:59
1931,cleaning,561,00:04:34
1963,toilet,571,00:00:41
2022,cleaning,581,00:02:16


In [43]:
index_of_entertainguests=df[df ["action"] == "entertainguests"].index.to_list()

vorgänger_entertainguests=[]
for i in index_of_entertainguests:
    vorgänger_entertainguests.append(i-1)
    
nachfolger_entertainguests=[]
for i in index_of_entertainguests:
    nachfolger_entertainguests.append(i+1)

df.iloc[vorgänger_entertainguests]
df.iloc[nachfolger_entertainguests]

,action,id,duration
1112,outdoors,341,01:34:18
1933,snack,561,03:06:51
2088,outdoors,591,02:22:35
2090,work,591,00:00:14


In [44]:
index_of_groom=df[df ["action"] == "groom"].index.to_list()

vorgänger_groom=[]
for i in index_of_groom:
    vorgänger_groom.append(i-1)
    
nachfolger_groom=[]
for i in index_of_groom:
    nachfolger_groom.append(i+1)

df.iloc[vorgänger_groom]
df.iloc[nachfolger_groom]

,action,id,duration
43,snack,21,00:05:56
217,mealpreperation,81,00:05:07
274,outdoors,91,00:10:16
282,outdoors,91,00:32:40
352,watchtv,101,00:48:52
560,sleep,161,02:00:09
574,relax,171,00:01:45
593,watchtv,171,00:00:02
636,snack,201,00:20:07
688,watchtv,211,00:07:51


In [45]:
index_of_mealpreperation=df[df ["action"] == "mealpreperation"].index.to_list()

vorgänger_mealpreperation=[]
for i in index_of_mealpreperation:
    vorgänger_mealpreperation.append(i-1)
    
nachfolger_mealpreperation=[]
for i in index_of_mealpreperation:
    nachfolger_mealpreperation.append(i+1)

df.iloc[vorgänger_mealpreperation]
df.iloc[nachfolger_mealpreperation]

,action,id,duration
21,watchtv,11,00:04:52
40,eatingdrinking,21,00:02:14
53,work,21,00:55:08
70,eatingdrinking,31,00:01:11
78,work,31,00:09:15
...,...,...,...
1930,eatingdrinking,561,00:08:52
2021,eatingdrinking,581,00:00:02
2038,toilet,581,00:09:28
2040,work,581,00:11:26


In [46]:
index_of_medication=df[df ["action"] == "medication"].index.to_list()

vorgänger_medication=[]
for i in index_of_medication:
    vorgänger_medication.append(i-1)
    
nachfolger_medication=[]
for i in index_of_medication:
    nachfolger_medication.append(i+1)

df.iloc[vorgänger_medication]
df.iloc[nachfolger_medication]

,action,id,duration
42,groom,21,00:13:47
57,work,21,00:00:34
73,work,31,00:30:49
95,watchtv,31,00:12:35
108,eatingdrinking,61,00:01:11
...,...,...,...
1990,sleep,571,00:57:36
2001,sleep,571,01:43:26
2017,dress,581,00:00:14
2072,mealpreperation,591,00:00:01


In [47]:
index_of_outdoors=df[df ["action"] == "outdoors"].index.to_list()

vorgänger_outdoors=[]
for i in index_of_outdoors:
    vorgänger_outdoors.append(i-1)
    
nachfolger_outdoors=[]
for i in index_of_outdoors:
    nachfolger_outdoors.append(i+1)

df.iloc[nachfolger_outdoors]

,action,id,duration
10,watchtv,11,00:47:11
14,work,11,00:28:30
20,mealpreperation,11,00:02:19
23,watchtv,11,00:09:38
25,watchtv,11,04:14:29
...,...,...,...
1967,personalhygiene,571,00:06:20
1982,personalhygiene,571,00:03:12
1993,toilet,571,00:04:40
2019,toilet,581,00:03:52


In [48]:
index_of_personalhygiene=df[df ["action"] == "personalhygiene"].index.to_list()

vorgänger_personalhygiene=[]
for i in index_of_personalhygiene:
    vorgänger_personalhygiene.append(i-1)
    
nachfolger_personalhygiene=[]
for i in index_of_personalhygiene:
    nachfolger_personalhygiene.append(i+1)

df.iloc[nachfolger_personalhygiene]

,action,id,duration
2,sleep,11,00:00:44
122,cleaning,61,00:01:49
124,mealpreperation,61,00:04:39
129,snack,61,00:14:55
164,mealpreperation,71,00:01:40
...,...,...,...
2029,cleaning,581,00:12:42
2055,End,581,00:00:00
2071,medication,591,00:00:01
2076,relax,591,00:18:48


In [49]:
index_of_phone=df[df ["action"] == "phone"].index.to_list()

vorgänger_phone=[]
for i in index_of_phone:
    vorgänger_phone.append(i-1)
    
nachfolger_phone=[]
for i in index_of_phone:
    nachfolger_phone.append(i+1)

df.iloc[nachfolger_phone]

,action,id,duration
82,outdoors,31,00:45:13
120,watchtv,61,00:21:25
132,watchtv,61,00:01:39
134,work,61,00:00:31
490,outdoors,151,00:40:10
744,toilet,221,00:03:23
765,work,231,00:17:08
920,outdoors,281,01:08:56
955,eatingdrinking,291,00:38:10
1466,work,431,00:00:35


In [50]:
index_of_relax=df[df ["action"] == "relax"].index.to_list()

vorgänger_relax=[]
for i in index_of_relax:
    vorgänger_relax.append(i-1)
    
nachfolger_relax=[]
for i in index_of_relax:
    nachfolger_relax.append(i+1)

df.iloc[nachfolger_relax]

,action,id,duration
62,watchtv,21,02:35:18
64,watchtv,21,01:10:12
86,snack,31,00:08:37
559,groom,161,00:22:09
575,mealpreperation,171,00:03:34
610,sleep,171,00:54:14
746,personalhygiene,221,00:01:21
797,toilet,231,00:20:13
877,personalhygiene,271,00:05:50
1006,mealpreperation,301,00:02:41


In [51]:
index_of_sleep=df[df ["action"] == "sleep"].index.to_list()

vorgänger_sleep=[]
for i in index_of_sleep:
    vorgänger_sleep.append(i-1)
    
nachfolger_sleep=[]
for i in index_of_sleep:
    nachfolger_sleep.append(i+1)

df.iloc[nachfolger_sleep]

,action,id,duration
3,toilet,11,01:53:41
5,eatingdrinking,11,00:11:23
17,snack,11,00:02:12
33,watchtv,21,00:00:58
36,work,21,02:45:09
...,...,...,...
2066,toilet,591,00:12:55
2068,toilet,591,00:00:15
2083,work,591,01:04:20
2096,toilet,591,00:06:41


In [52]:
index_of_snack=df[df ["action"] == "snack"].index.to_list()

vorgänger_snack=[]
for i in index_of_snack:
    vorgänger_snack.append(i-1)
    
nachfolger_snack=[]
for i in index_of_snack:
    nachfolger_snack.append(i+1)

df.iloc[nachfolger_snack]

,action,id,duration
12,work,11,00:23:36
18,dress,11,00:36:44
44,eatingdrinking,21,00:22:10
46,work,21,00:24:40
48,outdoors,21,00:33:49
60,eatingdrinking,21,00:05:19
76,outdoors,31,01:08:31
84,work,31,00:07:16
87,work,31,00:05:55
90,eatingdrinking,31,00:00:40


In [53]:
index_of_toilet=df[df ["action"] == "toilet"].index.to_list()

vorgänger_toilet=[]
for i in index_of_toilet:
    vorgänger_toilet.append(i-1)
    
nachfolger_toilet=[]
for i in index_of_toilet:
    nachfolger_toilet.append(i+1)

df.iloc[nachfolger_toilet]

,action,id,duration
4,sleep,11,00:03:48
35,sleep,21,01:52:29
97,watchtv,31,00:00:11
126,mealpreperation,61,00:02:02
136,work,61,00:00:58
...,...,...,...
2081,toilet,591,00:00:26
2082,sleep,591,00:43:59
2086,dress,591,00:01:01
2097,sleep,591,00:17:31


In [54]:
index_of_watchtv=df[df ["action"] == "watchtv"].index.to_list()

vorgänger_watchtv=[]
for i in index_of_watchtv:
    vorgänger_watchtv.append(i-1)
    
nachfolger_watchtv=[]
for i in index_of_watchtv:
    nachfolger_watchtv.append(i+1)

df.iloc[nachfolger_watchtv]

,action,id,duration
9,outdoors,11,00:29:23
11,snack,11,00:13:07
16,sleep,11,00:12:37
22,outdoors,11,00:16:49
24,outdoors,11,01:20:18
...,...,...,...
2042,toilet,581,00:00:56
2044,sleep,581,00:00:03
2046,work,581,00:26:02
2048,toilet,581,00:04:52


In [55]:
index_of_work=df[df ["action"] == "work"].index.to_list()

vorgänger_work=[]
for i in index_of_work:
    vorgänger_work.append(i-1)
    
nachfolger_work=[]
for i in index_of_work:
    nachfolger_work.append(i+1)

df.iloc[nachfolger_work]

,action,id,duration
13,outdoors,11,02:48:28
15,watchtv,11,00:46:01
37,dress,21,00:01:47
39,mealpreperation,21,00:01:42
47,snack,21,00:01:21
...,...,...,...
2047,watchtv,581,00:31:44
2054,personalhygiene,581,00:28:20
2080,toilet,591,00:00:39
2084,relax,591,00:31:29


In [56]:
list_bathe = {"Aktivität": "Bathe", "Vorgänger": [vorgänger_bathe], "Nachfolger": [nachfolger_bathe]}





In [57]:
list_cleaning = {"Aktivität": "Cleaning", "Vorgänger": [vorgänger_cleaning], "Nachfolger": [nachfolger_cleaning]}

In [58]:
list_dress = {"Aktivität": "Dress", "Vorgänger": [vorgänger_dress], "Nachfolger": [nachfolger_dress]}

In [59]:
list_eatingdrinking = {"Aktivität": "EatingDrinking", "Vorgänger": [vorgänger_eatingdrinking], "Nachfolger": [nachfolger_eatingdrinking]}

In [60]:
list_entertainguests = {"Aktivität": "EntertainGuests", "Vorgänger": [vorgänger_entertainguests], "Nachfolger": [nachfolger_entertainguests]}

In [61]:
list_groom = {"Aktivität": "Groom", "Vorgänger": [vorgänger_groom], "Nachfolger": [nachfolger_groom]}

In [62]:
list_phone = {"Aktivität": "Phone", "Vorgänger": [vorgänger_phone], "Nachfolger": [nachfolger_phone]}

In [63]:
list_mealpreperation = {"Aktivität": "Mealpreperation", "Vorgänger": [vorgänger_mealpreperation], "Nachfolger": [nachfolger_mealpreperation]}

In [64]:
list_medication = {"Aktivität": "Medication", "Vorgänger": [vorgänger_medication], "Nachfolger": [nachfolger_medication]}

In [65]:
list_outdoors = {"Aktivität": "Outdoors", "Vorgänger": [vorgänger_outdoors], "Nachfolger": [nachfolger_outdoors]}

In [66]:
list_personalhygiene = {"Aktivität": "PersonalHygiene", "Vorgänger": [vorgänger_personalhygiene], "Nachfolger": [nachfolger_personalhygiene]}

In [67]:
list_relax = {"Aktivität": "Relax", "Vorgänger": [vorgänger_relax], "Nachfolger": [nachfolger_relax]}

In [68]:
list_sleep = {"Aktivität": "Sleep", "Vorgänger": [vorgänger_sleep], "Nachfolger": [nachfolger_sleep]}

In [69]:
list_snack = {"Aktivität": "Snack", "Vorgänger": [vorgänger_snack], "Nachfolger": [nachfolger_snack]}

In [70]:
list_toilet = {"Aktivität": "Toilet", "Vorgänger": [vorgänger_toilet], "Nachfolger": [nachfolger_toilet]}

In [71]:
list_watchtv = {"Aktivität": "WatchTV", "Vorgänger": [vorgänger_watchtv], "Nachfolger": [nachfolger_watchtv]}

In [72]:
list_work = {"Aktivität": "Work", "Vorgänger": [vorgänger_work], "Nachfolger": [nachfolger_work]}

In [73]:
df_vorgänger_nachfolger = pd.DataFrame([list_bathe, list_cleaning, list_dress, list_eatingdrinking, list_entertainguests, list_groom,  list_phone, list_mealpreperation, list_medication,list_outdoors, list_personalhygiene, list_relax, list_sleep, list_snack, list_toilet, list_watchtv, list_work], columns = ['Aktivität','Vorgänger', 'Nachfolger'])

df_vorgänger_nachfolger

,Aktivität,Vorgänger,Nachfolger
0,Bathe,"[[1085, 1255, 1301, 1365, 1573, 1608, 1682, 18...","[[1087, 1257, 1303, 1367, 1575, 1610, 1684, 18..."
1,Cleaning,"[[5, 29, 70, 108, 121, 144, 166, 187, 252, 277...","[[7, 31, 72, 110, 123, 146, 168, 189, 254, 279..."
2,Dress,"[[6, 17, 26, 36, 1044, 1062, 1077, 1086, 1140,...","[[8, 19, 28, 38, 1046, 1064, 1079, 1088, 1142,..."
3,EatingDrinking,"[[4, 25, 39, 43, 59, 69, 78, 89, 100, 107, 115...","[[6, 27, 41, 45, 61, 71, 80, 91, 102, 109, 117..."
4,EntertainGuests,"[[1110, 1931, 2086, 2088]]","[[1112, 1933, 2088, 2090]]"
5,Groom,"[[41, 215, 272, 280, 350, 558, 572, 591, 634, ...","[[43, 217, 274, 282, 352, 560, 574, 593, 636, ..."
6,Phone,"[[80, 118, 130, 132, 488, 742, 763, 918, 953, ...","[[82, 120, 132, 134, 490, 744, 765, 920, 955, ..."
7,Mealpreperation,"[[19, 38, 51, 68, 76, 105, 114, 123, 125, 137,...","[[21, 40, 53, 70, 78, 107, 116, 125, 127, 139,..."
8,Medication,"[[40, 55, 71, 93, 106, 149, 188, 255, 278, 328...","[[42, 57, 73, 95, 108, 151, 190, 257, 280, 330..."
9,Outdoors,"[[8, 12, 18, 21, 23, 47, 49, 75, 81, 112, 117,...","[[10, 14, 20, 23, 25, 49, 51, 77, 83, 114, 119..."


In [74]:
df_vorgänger_nachfolger.to_excel("Python Table.xlsx")